In [3]:
# Hidden TimeStamp
import time, datetime
st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
print('Last Run: {}'.format(st))

Last Run: 2016-09-16 22:50:40


# Documenting Code

A general guide for authors and developers.

## As a Developer

This pertains to maintainers of the project repository.  Documentation is generated useing [Sphinx](http://www.sphinx-doc.org/en/stable/), hosted on [readthedocs](https://readthedocs.org/), the enhanced with [nbsphinx](https://nbsphinx.readthedocs.org/en/0.2.2/) and [conda](https://read-the-docs.readthedocs.io/en/latest/conda.html) to produce efficient builds. These tools give simple documentating experience by directly rendering jupyter notebooks on a hosted site.  Some details are written here for reference.

### Setup

The critical docs file structure is present below:

```
|+ docs
|    | 
|    |+ _images
|    |+ generated
|    |- README.md
|    |- index.rst
|    |- api.rst
|    |- conf.py
|    |- Makefile
|    |- make.bat
|    |- nbval_sanitize.cfg
|    |- environment.yml
|    |- *.ipynb
|    |- ...
|
|-readthedocs.yml

```

The `readthedocs.yml` and `environment.yml` files store metadata to make fast builds on readthedocs.  The folders prepended with and underscore are private and include pictures and data.  The `conf.py`, and `make` files are Sphinx files.  The `generated` contains Sphinx files that build an api tree.  The `nbval_sanitize.cfg` files has regexes used with `nbval` to clean output and perform regression tests (added in 0.4.13).  The remaining files are documents that are actively edited.  


### Building with conda

Building with large dependencies (numpy, pandas, matplotlib) can take a long time using `pip`.  Exceeding the build time limit (900 seconds) can cause builds to fail.  Therefore, we [build with conda](http://read-the-docs.readthedocs.io/en/latest/conda.html), which dramatically improves the build speed.  We begin by the mentioned yaml files in the appropriate locations.  Here are examples of each file:

```
# readthedocs.yml

conda:
    file: docs/environment.yml

python:
	version: 3
```

```
# environment.yml

name: lamana_docs
dependencies:
- python=3.5                                               # was installing py27
- matplotlib                                               # seems needed for sphinx on rtd
- pandas                                                   # speeds up build
- numpy
- openpyxl
- six
- ipykernel                                                # moved to conda 0.4.13-dev
- sphinx_rtd_theme                                         # added 0.4.13-dev
- pip:                                                     # unpinned 0.4.13-dev
  - nbsphinx
  - numpydoc
```

These files should trigger builds using conda.


### Using `nbsphinx`

Jupyter notebooks dwell in the "docs" folder.  By adding the `nbsphinx` extension to `conf.py`, notebooks extant in this folder are automatically converted from ipynb to html.  Specifically, running the `make html` build command does this conversion.  This setup has several benefits:

1. Edit notebooks directly; no copies or moves required from twin files.
2. Notebooks are rendered as-is.
3. Timestamps and command line info can be "hidden" prior to rendering (edit the metadata).
4. Images can pull from a single directory.

Only the `index.rst` file uses the native reST format.

### Images

Images for the entire package reside in the `./docs/_images` folder.  This placement enables root access to images for any notebook.  There is an `_archive` folder used  to store older versions of image files.  The `README` in the docs folder reminds us not to enumerate updated image files, otherwise notebook links will break. Rather, copy and enumerate the old file and archive for reference.

### API Docs

The [sphinx.ext.autosummary](http://www.sphinx-doc.org/en/stable/ext/autosummary.html) documentation is followed closely to build the API Reference page.  `cd` into the root package and run this code to update the API reference.

    > cd <package root>
    > sphinx-apidoc -f -o ./docs/generated .

This will create api docs (specifically "stubs") for all modules found in your package and store them in the "generated" folder.  These files extablish links between objects mentioned in your docs to their official docstrings. 

You can now clean the existing build folder and make a new html build for review before deployment.  

    > cd docs
    > make clean
    > make html

Open the `index.html` file in the `_build/html` folder.

The sphinx extension `viewcode` links to the exact code where each module is documented in the API reference.  The alternative is to use the "View in GitHub" link on each page in readthedocs (not observed locally).